In [0]:
# Importar bibliotecas necesarias
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Cargamos el conjunto de datos
df = pd.read_csv("https://raw.githubusercontent.com/Fibovin/des_modelos_1/refs/heads/main/Student_Performance.csv")

# Codificamos la variable categórica
df['Extracurricular Activities'] = df['Extracurricular Activities'].map({'Yes': 1, 'No': 0})

# Dividimos el conjunto de datos en características y variable objetivo
X = df.drop(columns=['Performance Index'])
y = df['Performance Index']

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuramos el experimento de MLflow
mlflow.set_experiment("/Users/jf.neiraf1@uniandes.edu.co/Student_Performance_SVR")

# Definimos y ejecutamos el experimento con MLflow para el modelo SVR
with mlflow.start_run(run_name="REGRESION SVR Experimento_1"):
    # Creamos y entrenamos el modelo SVR con kernel lineal
    svr_model = SVR(kernel="linear")
    svr_model.fit(X_train, y_train)

    # Realizamos predicciones en el conjunto de prueba
    y_test_pred_svr = svr_model.predict(X_test)

    # Calculamos y registramos las métricas para el conjunto de prueba
    r2_test_svr = r2_score(y_test, y_test_pred_svr)
    mse_test_svr = mean_squared_error(y_test, y_test_pred_svr)
    mae_test_svr = mean_absolute_error(y_test, y_test_pred_svr)
    
    mlflow.log_metric("r2", r2_test_svr)
    mlflow.log_metric("mse", mse_test_svr)
    mlflow.log_metric("mae", mae_test_svr)

    # Registramos el modelo SVR en MLflow
    mlflow.sklearn.log_model(svr_model, "svr-linear-model")

    # Imprimimos las métricas para verificar
    print("Conjunto de Prueba - SVR:")
    print(f"R²: {r2_test_svr}")
    print(f"Mean Squared Error: {mse_test_svr}")
    print(f"Mean Absolute Error: {mae_test_svr}")


2024/11/10 02:23:17 INFO mlflow.tracking.fluent: Experiment with name '/Users/jf.neiraf1@uniandes.edu.co/Student_Performance_SVR' does not exist. Creating a new experiment.
/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/11/10 02:23:32 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Conjunto de Prueba - SVR:
R²: 0.9889779818430801
Mean Squared Error: 4.084595877236296
Mean Absolute Error: 1.6117598399098356


## **Tunning del modelo - Regresión SVR**

### **Métodos de Ensamble**

In [0]:
# Configuramos el experimento de MLflow
mlflow.set_experiment("/Users/jf.neiraf1@uniandes.edu.co/Student_Performance_SVR")

# Definimos y ejecutamos el experimento con MLflow para el modelo de ensamble
with mlflow.start_run(run_name="REGRESION SVR ENSAMBLE Experimento_2"):
    # Creamos el modelo de ensamble con SVR y regresión lineal
    svr = SVR(C=1.0, kernel='rbf')
    lr = LinearRegression()
    ensemble_model = VotingRegressor([('svr', svr), ('lr', lr)])
    ensemble_model.fit(X_train, y_train)

    # Realizamos predicciones en el conjunto de prueba
    y_test_pred_ensemble = ensemble_model.predict(X_test)

    # Calculamos y registramos las métricas para el conjunto de prueba
    r2_test_ensemble = r2_score(y_test, y_test_pred_ensemble)
    mse_test_ensemble = mean_squared_error(y_test, y_test_pred_ensemble)
    mae_test_ensemble = mean_absolute_error(y_test, y_test_pred_ensemble)
    
    mlflow.log_metric("r2", r2_test_ensemble)
    mlflow.log_metric("mse", mse_test_ensemble)
    mlflow.log_metric("mae", mae_test_ensemble)

    # Registramos el modelo de ensamble en MLflow
    mlflow.sklearn.log_model(ensemble_model, "ensemble-svr-lr-model")

    # Imprimimos las métricas para verificar
    print("Conjunto de Prueba - Modelo de Ensamble (SVR y Regresión Lineal):")
    print(f"R²: {r2_test_ensemble}")
    print(f"Mean Squared Error: {mse_test_ensemble}")
    print(f"Mean Absolute Error: {mae_test_ensemble}")

2024/11/10 02:35:52 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Conjunto de Prueba - Modelo de Ensamble (SVR y Regresión Lineal):
R²: 0.9880481130744191
Mean Squared Error: 4.4291914027352925
Mean Absolute Error: 1.670586367787048
